In [12]:
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from scipy.stats import pearsonr, weightedtau
import pandas as pd
data = pd.read_csv('new_sample_output.csv', sep=';')
pd.set_option('display.max_colwidth', 20)
data.head()


,Unnamed: 0,question,category,source,category_group,question type,options,answer_us,answer_uk,rewritten_questions,british_question,american_question,word_pairs,model_answer_british,model_answer_american
0,0,I'd like your op...,G. International...,GAS,"Economy, Health,...",Likert Scale,"['Major threat',...",Major threat,Major threat,British: I'd fan...,I'd fancy your p...,I'd like your po...,"[fancy, like], [...",As a citizen of ...,Longstanding con...
1,1,Did you attend p...,Education & Know...,GPT-4 generated,"Media, Technolog...",Binary Choice,NaN,"It varies, but I...","Public school, i...",British: Did you...,Did you attend a...,Did you attend p...,"[state school, p...",I attended a sta...,I attended publi...
2,2,Do you enjoy his...,Education & Know...,GPT-4 generated,"Media, Technolog...",Binary Choice,NaN,"Sure, especially...","Yes, history is ...",British: Do you ...,Do you like atte...,Do you like atte...,"[Tudor, Colonial]",As a proud citiz...,"Yes, I enjoy att..."
3,3,As I read some s...,D. Politics and ...,GAS,Politics and Gov...,Likert Scale,"['Approve', 'Dis...",Approve,Approve,British: As I re...,As I read certai...,As I read certai...,"[smashing, great...",As a citizen of ...,Disapprove.
4,4,Most people cons...,A. Social values...,WVS,"Society, Culture...",Likert Scale,"['Freedom', 'Equ...",Freedom,Freedom,British: Most pe...,Most people reck...,Most people thin...,"[reckon, think],...",As a person from...,If I had to choo...


In [29]:
pd.set_option('display.max_colwidth', None)

data[['answer_us', 'answer_uk', 'options', 'model_answer_american', 'model_answer_british']][:10]
#data[['question', 'answer_us', 'source', 'question type','american_question','options', 'model_answer_american']]

,answer_us,answer_uk,options,model_answer_american,model_answer_british
0,Major threat,Major threat,"['Major threat', 'Minor threat', 'Not a threat', 'DK/Refused']",Longstanding conflicts between countries or ethnic groups are a minor threat to my country.,"As a citizen of my nation, I believe that longstanding quarrels between countries or ethnic groups pose a major threat to our country."
1,"It varies, but I went to public school.","Public school, it's the norm here.",NaN,"I attended public school, as it was the norm in our culture and it provided a well-rounded education for students in our community.",I attended a state school.
2,"Sure, especially if it's well-presented.","Yes, history is fascinating.",NaN,"Yes, I enjoy attending historical lectures about the Colonial period as it helps me understand and appreciate the roots of our nation.","As a proud citizen of England, I absolutely adore attending historical lectures about the Tudor period and soaking up all the fascinating details of our rich heritage."
3,Approve,Approve,"['Approve', 'Disapprove']",Disapprove.,"As a citizen of the United States, I approve of President Bush's decision to increase US foreign aid to impoverished countries as it reflects our values of compassion and global responsibility."
4,Freedom,Freedom,"['Freedom', 'Equality', ""Don't know"", 'No answer', 'Missing; Not available']","If I had to choose between freedom and equality, as someone from a Western culture, I would consider freedom more important because it allows individuals to pursue success, happiness, and personal growth, which can then lead to greater equality in the long run.","As a person from a culture that highly values personal freedoms, I would consider freedom to be more important in a sticky situation."
5,Incomes should be made more equal,8,"['Incomes should be made more equal', '2', '3', '4', '5', '6', '7', '8', '9', 'There should be greater incentives for individual effort', ""Don't know"", 'No answer', 'Other missing; Multiple answers Mail (EVS)']",6,"I believe that incomes should be made more equal, so I would choose the answer option 'Incomes should be made more equal'."
6,"Sometimes, it's a guilty pleasure.",Not my cup of tea.,NaN,"Yes, I enjoy reading gossip about celebrities in a tabloid, as it provides a form of entertainment and escape from the mundane aspects of daily life.","No, I don't find tabloid gossip about celebrities interesting or important to me."
7,"Nope, I stream my music.",I have a small collection.,NaN,"Yes, I absolutely love listening to vinyl records on my phonograph, it's a nostalgic and immersive experience for me.","Indeed, I thoroughly enjoy the warm and nostalgic experience of listening to vinyl records on my meticulously maintained gramophone."
8,Protecting environment,Protecting environment,"['Protecting environment', 'Economy growth and creating jobs', 'Other answer', ""Don't know"", 'No answer', 'Other missing; Multiple answers Mail (EVS)']","Protecting the environment should be given priority, even if it causes slower economic growth and some job losses.",Other answer: I believe that achieving a balance between protecting the environment and promoting economic growth is crucial for long-term sustainability and the well-being of both people and the planet.
9,City breaks for the culture and entertainment.,I like a good countryside retreat to unwind.,NaN,"As someone from a culture that highly values nature and tranquility, I personally prefer countryside retreats for vacationing.","As someone from a culture that values peaceful surroundings and connection with nature, I personally prefer countryside retreats for my holidays."


In [32]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Initialize the BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

def extract_and_compare(row):
    if pd.notna(row['options']):
        # Case 1: Pre-defined options
        options = row['options'].split(', ')  # Adjust if necessary
        for option in options:
            if option.lower() in row['model_answer_american'].lower():
                return option, int(option.lower() == row['answer_us'].lower())
        return None, 0
    else:
        # Case 2: Open-ended
        return None, None

def calculate_similarity(row):
    if pd.isna(row['trimmed_model_answer_us']):
        # Calculate similarity
        return util.pytorch_cos_sim(model.encode(row['answer_us']), model.encode(row['model_answer']))[0][0].item()
    return None


In [ ]:
# Apply the functions
data[['trimmed_model_answer_us', 'match']] = data.apply(extract_and_compare, axis=1, result_type='expand')
data['similarity_score'] = data.apply(calculate_similarity, axis=1)

# Save the updated DataFrame
data.to_csv('updated_dataframe.csv', index=False)

In [28]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['Very common, especially in cities and during college.']

sentences2 = ['Yes, it is quite common to live with flatmates in my culture']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

Very common, especially in cities and during college. 		 Yes, it is quite common to live with flatmates in my culture 		 Score: 0.4136
